In [81]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import sys

In [82]:
def clean(img_dir, save_dir, img_name):
    # read image
    img = cv2.imread(join(img_dir, img_name))

    # convert to LAB and extract L  channel
    LAB = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L = LAB[:,:,0]

    # threshold L channel with triangle method
    value, thresh = cv2.threshold(L, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_TRIANGLE)
    # print(value)

    value = value + 10
    thresh = cv2.threshold(L, value, 255, cv2.THRESH_BINARY)[1]

    # invert threshold and make 3 channels
    thresh = 255 - thresh
    mask = cv2.merge([thresh, thresh, thresh])

    mask = cv2.blur(mask, (100, 100), 0)

    a = 0.83
    res = cv2.addWeighted(img, a,mask,1 - a,0)
    cv2.imwrite(join(save_dir, img_name), res)
    # cv2.imwrite("clear_data/photo/out3.jpg", res)

In [83]:
from os import listdir
from os.path import isfile, join

t = 10
# for f in listdir("dataset/photo"):
    # clean("dataset\photo", "clear_data\photo", f)
    # t -= 1
    # if not t:
    #     break

# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [84]:
path = "clear_data/photo"
first = listdir(path)[0]
second = listdir(path)[1]
img1 = cv2.imread(join(path, first), cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread(join(path, second), cv2.IMREAD_GRAYSCALE)

# cv2.imshow("img1", img1)
# cv2.imshow("img2", img2)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [85]:
# Create our ORB detector and detect keypoints and descriptors
orb = cv2.ORB_create(nfeatures=2000)

# Find the key points and descriptors with ORB
keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

In [86]:
# cv2.imshow("Key_points1", cv2.drawKeypoints(img1, keypoints1, None, (255, 0, 255)))
# cv2.imshow("Key_points2", cv2.drawKeypoints(img2, keypoints2, None, (255, 0, 255)))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [87]:
# Create a BFMatcher object.
# It will find all of the matching keypoints on two images
bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)

# Find matching points
matches = bf.knnMatch(descriptors1, descriptors2,k=2)

In [88]:
def draw_matches(img1, keypoints1, img2, keypoints2, matches):
  r, c = img1.shape[:2]
  r1, c1 = img2.shape[:2]

  # Create a blank image with the size of the first image + second image
  output_img = np.zeros((max([r, r1]), c+c1, 3), dtype='uint8')
  output_img[:r, :c, :] = np.dstack([img1, img1, img1])
  output_img[:r1, c:c+c1, :] = np.dstack([img2, img2, img2])

  # Go over all of the matching points and extract them
  for match in matches:
    img1_idx = match.queryIdx
    img2_idx = match.trainIdx
    (x1, y1) = keypoints1[img1_idx].pt
    (x2, y2) = keypoints2[img2_idx].pt

    # Draw circles on the keypoints
    cv2.circle(output_img, (int(x1),int(y1)), 4, (0, 255, 255), 1)
    cv2.circle(output_img, (int(x2)+c,int(y2)), 4, (0, 255, 255), 1)

    # Connect the same keypoints
    cv2.line(output_img, (int(x1),int(y1)), (int(x2)+c,int(y2)), (0, 255, 255), 1)

  return output_img

In [89]:
all_matches = []
for m, n in matches:
  all_matches.append(m)

img3 = draw_matches(img1, keypoints1, img2, keypoints2, all_matches[:30])
# cv2.imshow("output", img3)
#
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [90]:
# Finding the best matches
good = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good.append(m)

In [91]:
# cv2.imshow("goodKeys1", cv2.drawKeypoints(img1, [keypoints1[m.queryIdx] for m in good], None, (255, 0, 255)))
# cv2.imshow("goodKeys2", cv2.drawKeypoints(img2, [keypoints2[m.trainIdx] for m in good], None, (255, 0, 255)))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [92]:
def warpImages(img1, img2, H):

  rows1, cols1 = img1.shape[:2]
  rows2, cols2 = img2.shape[:2]

  list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2)
  temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)

  # When we have established a homography we need to warp perspective
  # Change field of view
  list_of_points_2 = cv2.perspectiveTransform(temp_points, H)

  list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)

  [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
  [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)

  translation_dist = [-x_min,-y_min]

  H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])

  output_img = cv2.warpPerspective(img2, H_translation.dot(H), (x_max-x_min, y_max-y_min))
  output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1

  return output_img

In [93]:
# Set minimum match condition
MIN_MATCH_COUNT = 10

if len(good) > MIN_MATCH_COUNT:
    # Convert keypoints to an argument for findHomography
    src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

    # Establish a homography
    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

    result = warpImages(img2, img1, M)
    # cv2.imshow("result", result)
    #
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [162]:
def main(img1, img2):
    # Create our ORB detector and detect keypoints and descriptors
    orb = cv2.ORB_create(nfeatures=2000)

    # Find the key points and descriptors with ORB
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    # Create a BFMatcher object.
    # It will find all of the matching keypoints on two images
    bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)

    # Find matching points
    matches = bf.knnMatch(descriptors1, descriptors2,k=2)

    all_matches = []
    for m, n in matches:
      all_matches.append(m)

    good = []
    for m, n in matches:
        if m.distance < 0.6 * n.distance:
            good.append(m)

    # Set minimum match condition
    MIN_MATCH_COUNT = 10

    if len(good) > MIN_MATCH_COUNT:
        # Convert keypoints to an argument for findHomography
        src_pts = np.float32([ keypoints1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst_pts = np.float32([ keypoints2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        # Establish a homography
        M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)

        result = warpImages(img2, img1, M)

        return result
    else:
        return img1

In [166]:
path = "clear_data/photo"

start = 20
first = listdir(path)[start]
img1 = cv2.imread(join(path, first), cv2.IMREAD_GRAYSCALE)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
for i in range(start + 1, start + 30):
    second = listdir(path)[i]
    img2 = cv2.imread(join(path, second), cv2.IMREAD_GRAYSCALE)
    img1 = main(img1, img2)

cv2.imshow("res", img1)
cv2.imwrite("res.jpg", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()